In [51]:
import numpy as np
import pandas as pd
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('/Users/nurpeiis/Desktop/Capstone/arabic-did/')
import data_utils
import filter_logit
import finetuning_utils
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [13]:
labels, label2id, id2label = data_utils.get_label_space('../labels/city_label_id.txt')
madar_labels, madar_label2id, madar_id2label = data_utils.get_label_space('../labels/madar_city_label_id.txt')

In [11]:
madar_folder =  '../../hierarchical-did/data_processed_second/madar_shared_task1/'
test_file = f'{madar_folder}MADAR-Corpus-26-test.lines'
df = pd.read_csv(test_file, sep='\t', header=0)
true_labels = data_utils.extract_labels(df, 'city',  '../labels/city_label_id.txt')

In [12]:
predictions = np.load('../madar_city.npz')['arr_0']


In [16]:
predictions

array([[ 3.028184  , -1.2136688 , -1.0954336 , ..., -2.4738507 ,
        -3.158033  ,  1.1985927 ],
       [ 1.7041996 , -4.157486  , -1.3547418 , ..., -2.9284716 ,
        -3.392424  , -0.9316238 ],
       [-0.5834973 , -0.25821796, -1.2882706 , ..., -1.1921716 ,
        -1.9073467 , -0.41871026],
       ...,
       [ 0.9381845 , -0.9647397 , -1.0214208 , ..., -1.440158  ,
         0.40938205,  0.8588096 ],
       [ 1.9956946 , -1.9208399 ,  0.02428623, ..., -0.45730105,
        -0.4462226 , -1.5883727 ],
       [ 1.1452665 , -1.2395177 ,  0.26085114, ..., -1.1150992 ,
        -2.2947567 ,  1.8532526 ]], dtype=float32)

In [46]:
list_metrics = ['accuracy', 'precision', 'recall', 'f1', 'loss']
total_metrics = {}
for m in list_metrics:
    total_metrics[m] = 0
def metrics(pred_flat, labels_flat, total_metrics):
    f1 = f1_score(labels_flat, pred_flat, average='macro')
    precision = precision_score(labels_flat, pred_flat, average='macro')
    recall = recall_score(labels_flat, pred_flat, average='macro')
    accuracy = accuracy_score(labels_flat, pred_flat)

    list_metrics = ['accuracy', 'precision', 'recall', 'f1']
    cur_metrics = {'accuracy': accuracy,
                   'precision': precision, 'recall': recall, 'f1': f1}
    for m in list_metrics:
        total_metrics[m] += cur_metrics[m]
    return cur_metrics
def flatten_label(preds):
    labels = []
    for i in range(len(preds)):
        index = 0
        val = -99
        for j in range(len(preds[i])):
            if val < preds[i][j]:
                index = j
                val = preds[i][j] 
        labels.append(index)
    return labels


In [53]:
def get_space(aggregate_label_space_file, search_label_space_file):

    aggregate_labels, aggregate_label2id, aggregate_id2label = data_utils.get_label_space(
        aggregate_label_space_file)

    search_labels, search_label2id, search_id2label = data_utils.get_label_space(
        search_label_space_file)

    space = []
    for l in list(search_label2id.keys()):
        if l in aggregate_label2id:
            space.append(aggregate_label2id[l])

    return space

In [54]:
space = get_space('../labels/city_label_id.txt', '../labels/madar_city_label_id.txt')
null_labels = filter_logit.nullify_weight(predictions, space, -99.9)
print(null_labels)
for i in range(len(null_labels)):
    null_labels[i] = np.array(null_labels[i], dtype=np.float32)
null_labels = np.array(null_labels, dtype=np.float32)
null_preds = flatten_label(null_labels)
null_metrics = metrics(null_preds, true_labels,total_metrics)
null_metrics

[[  3.02818394 -99.9        -99.9        ... -99.9        -99.9
    1.19859266]
 [  1.70419955 -99.9        -99.9        ... -99.9        -99.9
   -0.93162382]
 [ -0.58349729 -99.9        -99.9        ... -99.9        -99.9
   -0.41871026]
 ...
 [  0.9381845  -99.9        -99.9        ... -99.9        -99.9
    0.85880959]
 [  1.99569464 -99.9        -99.9        ... -99.9        -99.9
   -1.58837271]
 [  1.14526653 -99.9        -99.9        ... -99.9        -99.9
    1.85325265]]


{'accuracy': 0.04326923076923077,
 'precision': 0.044149772839249035,
 'recall': 0.043269230769230775,
 'f1': 0.039021606817109374}

In [48]:
null_preds

[78,
 28,
 78,
 86,
 0,
 94,
 94,
 112,
 48,
 55,
 36,
 15,
 57,
 94,
 94,
 22,
 22,
 78,
 28,
 78,
 94,
 15,
 94,
 70,
 15,
 48,
 51,
 88,
 28,
 0,
 28,
 22,
 57,
 36,
 0,
 36,
 86,
 0,
 58,
 45,
 88,
 94,
 0,
 88,
 86,
 90,
 26,
 51,
 0,
 112,
 94,
 61,
 46,
 61,
 70,
 26,
 22,
 90,
 94,
 94,
 36,
 36,
 94,
 45,
 94,
 86,
 45,
 51,
 58,
 88,
 78,
 51,
 61,
 78,
 28,
 26,
 15,
 51,
 74,
 94,
 29,
 78,
 51,
 15,
 88,
 88,
 94,
 0,
 15,
 112,
 51,
 46,
 51,
 14,
 78,
 51,
 14,
 94,
 15,
 88,
 94,
 45,
 89,
 22,
 55,
 36,
 15,
 46,
 88,
 112,
 86,
 78,
 78,
 0,
 55,
 74,
 94,
 14,
 0,
 88,
 15,
 94,
 70,
 78,
 22,
 29,
 0,
 88,
 26,
 78,
 15,
 90,
 86,
 45,
 45,
 51,
 45,
 48,
 48,
 51,
 74,
 15,
 78,
 29,
 28,
 29,
 58,
 15,
 89,
 78,
 36,
 26,
 51,
 46,
 28,
 36,
 88,
 78,
 61,
 94,
 46,
 51,
 58,
 88,
 15,
 0,
 48,
 70,
 57,
 55,
 94,
 88,
 46,
 51,
 94,
 88,
 26,
 51,
 58,
 74,
 88,
 55,
 0,
 51,
 15,
 88,
 15,
 15,
 14,
 94,
 29,
 112,
 94,
 78,
 29,
 15,
 28,
 88,
 70,
 57,
 78,
 0

In [49]:
null_labels[0][78]

14.464212

In [50]:
id2label[78]

'rabat ma maghreb'